In [28]:
import pandas as pd
import numpy as np

In [4]:
df = pd.read_csv('geocoded_addresses.csv')

In [37]:
coords = tuple(zip(df['latitude'], df['longitude']))
def to_osrm_format(backwards_coords):
    return [(lon, lat) for lat, lon in coords]
coords = remove_outlier_coords(to_osrm_format(coords))
coords

removed 37 pairs


[(47.7220511, -122.35584),
 (47.7220511, -122.35584),
 (47.7216444, -122.3559837),
 (47.721630450000006, -122.35655207667556),
 (47.7211526, -122.3559906),
 (47.71873115, -122.35586081596126),
 (47.71809665, -122.35586685),
 (47.7157996, -122.35662351858495),
 (47.6329947, -122.3566878),
 (47.720153100000005, -122.35513650144014),
 (47.72045335, -122.35517063516772),
 (47.721994, -122.355096),
 (47.72305225, -122.35445684760458),
 (47.722384000000005, -122.35450517276536),
 (47.72191505, -122.35448839909049),
 (47.72163115, -122.35448784707656),
 (47.72160245, -122.35393559389024),
 (47.7211803, -122.35447095),
 (47.72096925, -122.35459650340908),
 (47.71901305, -122.35452790625132),
 (47.71800245, -122.35449899605302),
 (47.7175958, -122.3545337925087),
 (47.71721425, -122.35450472661827),
 (47.7167743, -122.35450897762942),
 (47.7153187, -122.3544718894101),
 (47.71481485, -122.3545386908126),
 (47.7139723, -122.35448930321),
 (47.7136208, -122.35450614617837),
 (47.71299195, -122.35

In [36]:
# for testing routing only, will find root cause for erroneoud coordinates later
def remove_outlier_coords(coords):
    min_lat=47.0
    max_lat=47.9 
    min_lon=-122.6
    max_lon=-122.0
    cleaned_coords = []
    
    for coord in coords:
        if min_lat <= coord[0] <= max_lat and min_lon <= coord[1] <= max_lon:
            cleaned_coords.append(coord)
    print(f'removed {len(coords) - len(cleaned_coords)} pairs')
    return cleaned_coords

In [21]:
import requests
import math
import logging # For basic feedback

# Configure basic logging if you want to see info/error messages
# You can run this once in a notebook cell if you haven't already
# logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)
# If you don't want to configure basicConfig, print statements will be used for errors.

def get_osrm_distance_matrix(coord_points_lat_lon, batch_size=50, osrm_port=8001, profile="driving"):
    """
    Calculates a full N x N all-to-all distance matrix for a list of coordinate points
    by batching requests to an OSRM server. Each point serves as both a potential
    source and a potential destination in the final matrix.

    Args:
        coord_points_lat_lon (list or tuple): 
            A list or tuple of coordinate pairs.
            Each pair MUST be in (latitude, longitude) format.
        batch_size (int): The number of coordinates to process in each dimension of a sub-request.
                          Max unique coordinates in one OSRM URL path will be up to 2*batch_size.
                          Default 50 (for 50x50 blocks, up to 100 unique points in combined requests).
        osrm_port (int): The port on localhost where the OSRM server is listening.
        profile (str): OSRM routing profile (e.g., "driving", "walking", "cycling").

    Returns:
        list: A list of lists representing the full N x N distance matrix (in meters).
              Returns None if a critical error occurs that prevents matrix completion.
    """
    num_total_points = len(coord_points_lat_lon)
    if num_total_points == 0:
        print("ERROR: No coordinate points provided.")
        logger.error("No coordinate points provided.")
        return []
    if num_total_points == 1:
        print("WARNING: Only one coordinate point provided. Distance matrix will be [[0.0]].")
        logger.warning("Only one coordinate point provided. Distance matrix will be [[0.0]].")
        return [[0.0]] 

    # Initialize the full N x N result matrix with None values
    # Using float for distances, None for uncomputed/error cells
    full_distance_matrix = [[None for _ in range(num_total_points)] for _ in range(num_total_points)]

    osrm_url_template = "http://localhost:{port}/table/v1/{profile}/{coordinates_for_url}"
    
    message = f"Starting batched OSRM all-to-all distance matrix for {num_total_points} points (batch size: {batch_size})."
    print(message)
    logger.info(message)

    # Iterate through source chunks
    for i_start in range(0, num_total_points, batch_size):
        i_end = min(i_start + batch_size, num_total_points)
        source_chunk_coords = coord_points_lat_lon[i_start:i_end]
        num_sources_in_current_chunk = len(source_chunk_coords)

        # Iterate through destination chunks
        for j_start in range(0, num_total_points, batch_size):
            j_end = min(j_start + batch_size, num_total_points)
            dest_chunk_coords = coord_points_lat_lon[j_start:j_end]
            num_dests_in_current_chunk = len(dest_chunk_coords)

            current_batch_info = (f"Processing source batch (points {i_start}-{i_end-1}) "
                                  f"vs destination batch (points {j_start}-{j_end-1})...")
            print(current_batch_info)
            logger.info(current_batch_info)

            # Prepare coordinates for this specific API call.
            # These are the actual points whose distances are being calculated in this batch.
            # OSRM's 'sources' and 'destinations' parameters will refer to indices within this list.
            api_call_coords_tuples = source_chunk_coords + dest_chunk_coords
            # OSRM is efficient with duplicate coordinates in the path if indices are distinct.

            if not api_call_coords_tuples:
                logger.warning("Empty coordinate list for current OSRM request batch. Skipping.")
                continue

            # Format coordinates for OSRM URL: "lon,lat;lon,lat;..."
            formatted_coords_list_for_url = [f"{lon},{lat}" for lat, lon in api_call_coords_tuples]
            coordinates_string_for_url = ";".join(formatted_coords_list_for_url)

            # Define source and destination indices *within the api_call_coords_tuples*
            source_indices_in_api_call = list(range(num_sources_in_current_chunk))
            dest_indices_in_api_call = list(range(num_sources_in_current_chunk, 
                                                  num_sources_in_current_chunk + num_dests_in_current_chunk))

            params = {
                "annotations": "distance",
                "sources": ";".join(map(str, source_indices_in_api_call)),
                "destinations": ";".join(map(str, dest_indices_in_api_call))
            }
            
            request_url = osrm_url_template.format(
                port=osrm_port,
                profile=profile,
                coordinates_for_url=coordinates_string_for_url
            )

            try:
                response = requests.get(request_url, params=params, timeout=30) # Increased timeout
                response.raise_for_status() 
                
                response_json = response.json()

                if response_json.get("code") == "Ok":
                    sub_matrix = response_json.get("distances")
                    # Validate dimensions of the returned sub_matrix
                    if sub_matrix and \
                       len(sub_matrix) == num_sources_in_current_chunk and \
                       all(len(row) == num_dests_in_current_chunk for row in sub_matrix):
                        
                        # Place the sub_matrix into the full_distance_matrix
                        for src_idx_in_chunk, row_data in enumerate(sub_matrix):
                            for dst_idx_in_chunk, distance in enumerate(row_data):
                                # Calculate final matrix indices
                                final_src_idx = i_start + src_idx_in_chunk
                                final_dst_idx = j_start + dst_idx_in_chunk
                                full_distance_matrix[final_src_idx][final_dst_idx] = distance
                        logger.debug(f"Successfully processed sub-matrix for sources {i_start}-{i_end-1} to dests {j_start}-{j_end-1}.")
                    else:
                        error_msg = (f"OSRM returned 'Ok' but 'distances' matrix is missing, null, or has "
                                     f"unexpected dimensions ({len(sub_matrix)}x{len(sub_matrix[0]) if sub_matrix and sub_matrix[0] else 'N/A'}) "
                                     f"for sources {i_start}-{i_end-1} (expected {num_sources_in_current_chunk}) "
                                     f"to dests {j_start}-{j_end-1} (expected {num_dests_in_current_chunk}).")
                        print(f"ERROR: {error_msg}")
                        logger.error(error_msg)
                        # Mark this block as failed in the main matrix
                        for r_idx in range(i_start, i_end):
                            for c_idx in range(j_start, j_end):
                                full_distance_matrix[r_idx][c_idx] = float('inf') # Or another error marker
                else:
                    error_code = response_json.get('code', 'Unknown Code')
                    api_error_message = response_json.get("message", "Unknown OSRM API error")
                    full_error_msg = (f"OSRM API error for sources {i_start}-{i_end-1} to dests {j_start}-{j_end-1}: "
                                      f"{error_code} - {api_error_message}")
                    print(f"ERROR: {full_error_msg}")
                    logger.error(full_error_msg)
                    for r_idx in range(i_start, i_end):
                        for c_idx in range(j_start, j_end):
                            full_distance_matrix[r_idx][c_idx] = float('inf')


            except requests.exceptions.RequestException as e:
                req_exc_msg = f"RequestException for sources {i_start}-{i_end-1} to dests {j_start}-{j_end-1}: {e}"
                print(f"ERROR: {req_exc_msg}")
                logger.error(req_exc_msg)
                for r_idx in range(i_start, i_end):
                    for c_idx in range(j_start, j_end):
                        full_distance_matrix[r_idx][c_idx] = float('inf') 
                # Optional: Abort all if one batch fails critically
                # print("Aborting due to critical batch failure.")
                # return None 

    message = "Finished all batched OSRM requests."
    print(message)
    logger.info(message)
    return full_distance_matrix

In [38]:
    target_osrm_port = 8001

    print(f"Requesting distance matrix on port {target_osrm_port}...")
    matrix = get_osrm_distance_matrix(coords, osrm_port=target_osrm_port)

    if matrix:
        print("\nDistance Matrix (meters):")
        for i, row in enumerate(matrix):
            formatted_row = [f"{dist:.0f}" if dist is not None else "N/A" for dist in row]
            print(f"Point {i}: {formatted_row}")
        
        if len(matrix) > 2 and matrix[0] is not None and len(matrix[0]) > 2 and matrix[0][2] is not None:
            print(f"\nDistance from Point 0 to Point 2: {matrix[0][2]/1000:.2f} km")
    else:
        print("\nCould not retrieve the distance matrix.")

2025-05-15 11:20:49,664 - INFO - __main__ - Starting batched OSRM all-to-all distance matrix for 666 points (batch size: 50).
2025-05-15 11:20:49,664 - INFO - __main__ - Processing source batch (points 0-49) vs destination batch (points 0-49)...
2025-05-15 11:20:49,750 - INFO - __main__ - Processing source batch (points 0-49) vs destination batch (points 50-99)...
2025-05-15 11:20:49,833 - INFO - __main__ - Processing source batch (points 0-49) vs destination batch (points 100-149)...


Requesting distance matrix on port 8001...
Starting batched OSRM all-to-all distance matrix for 666 points (batch size: 50).
Processing source batch (points 0-49) vs destination batch (points 0-49)...
Processing source batch (points 0-49) vs destination batch (points 50-99)...
Processing source batch (points 0-49) vs destination batch (points 100-149)...


2025-05-15 11:20:49,919 - INFO - __main__ - Processing source batch (points 0-49) vs destination batch (points 150-199)...
2025-05-15 11:20:50,004 - INFO - __main__ - Processing source batch (points 0-49) vs destination batch (points 200-249)...


Processing source batch (points 0-49) vs destination batch (points 150-199)...
Processing source batch (points 0-49) vs destination batch (points 200-249)...


2025-05-15 11:20:50,174 - INFO - __main__ - Processing source batch (points 0-49) vs destination batch (points 250-299)...
2025-05-15 11:20:50,266 - INFO - __main__ - Processing source batch (points 0-49) vs destination batch (points 300-349)...
2025-05-15 11:20:50,358 - INFO - __main__ - Processing source batch (points 0-49) vs destination batch (points 350-399)...


...

2025-05-15 11:21:08,993 - INFO - __main__ - Processing source batch (points 650-665) vs destination batch (points 400-449)...
2025-05-15 11:21:09,065 - INFO - __main__ - Processing source batch (points 650-665) vs destination batch (points 450-499)...
2025-05-15 11:21:09,134 - INFO - __main__ - Processing source batch (points 650-665) vs destination batch (points 500-549)...


Processing source batch (points 650-665) vs destination batch (points 400-449)...
Processing source batch (points 650-665) vs destination batch (points 450-499)...
Processing source batch (points 650-665) vs destination batch (points 500-549)...


2025-05-15 11:21:09,203 - INFO - __main__ - Processing source batch (points 650-665) vs destination batch (points 550-599)...
2025-05-15 11:21:09,271 - INFO - __main__ - Processing source batch (points 650-665) vs destination batch (points 600-649)...
2025-05-15 11:21:09,342 - INFO - __main__ - Processing source batch (points 650-665) vs destination batch (points 650-665)...
2025-05-15 11:21:09,386 - INFO - __main__ - Finished all batched OSRM requests.


Processing source batch (points 650-665) vs destination batch (points 550-599)...
Processing source batch (points 650-665) vs destination batch (points 600-649)...
Processing source batch (points 650-665) vs destination batch (points 650-665)...
Finished all batched OSRM requests.

Distance Matrix (meters):
Point 0: ['0', '0', '145', '145', '103', '388', '518', '811', '11357', '227', '208', '38', '252', '326', '235', '204', '200', '196', '165', '441', '549', '549', '633', '682', '844', '900', '954', '1035', '1105', '1120', '4189', '1322', '1768', '1452', '1470', '1505', '11357', '1656', '1689', '1717', '1734', '1746', '11357', '1359', '1416', '1462', '1252', '906', '954', '1061', '1161', '305', '451', '449', '503', '524', '616', '600', '661', '789', '737', '2219', '4719', '2196', '2197', '2190', '2129', '2103', '2105', '2050', '1985', '2001', '1971', '1747', '1736', '1780', '1736', '1682', '1712', '1599', '1560', '1506', '1472', '1414', '1561', '1556', '1339', '1930', '1939', '1844', '

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [17]:
def to_osrm_format(backwards_coords):
    return [(lon, lat) for lat, lon in coords]
coords = to_osrm_format(coords)
coords

In [39]:
new_df = pd.DataFrame(coords)
new_df.to_csv('coordinates.csv', index=False)
new_df

,0,1
0,47.722051,-122.355840
1,47.722051,-122.355840
2,47.721644,-122.355984
3,47.721630,-122.356552
4,47.721153,-122.355991
...,...,...
661,47.709597,-122.295245
662,47.709486,-122.295244
663,47.709311,-122.295147
664,47.709071,-122.295163


In [40]:
np_matrix = np.array(matrix)

In [41]:
np.save('distance_matrix.npy', np_matrix)